<br><center><span style="color: green; font-size: 40px; font-weight: bold;">Projeto 2 (Dados em Tempo Real)</span></center>

<br><br><br>

# <center>Análise de Sentimentos sobre o ChatGPT no Twitter</center>

<br>

### Contexto

Este projeto explora a análise de sentimentos de tweets sobre o ChatGPT para entender como o público está reagindo a essa nova tecnologia. Desde o lançamento do ChatGPT, as opiniões têm sido divididas, com alguns elogiando seu potencial, enquanto outros demonstram receio em relação à substituição de empregos e outras questões sociais. A análise de sentimentos é uma ferramenta essencial para medir essas reações.

Neste projeto, tweets **já foram extraídos em tempo real** da plataforma X (anteriormente conhecida como Twitter) utilizando o Bardeen AI. **No entanto, devido à limitação da opção gratuita do Bardeen AI**, não foi possível manter a extração contínua ao longo do tempo. Por isso, os dados utilizados na análise representam uma amostra de um determinado período, capturando as opiniões mais recentes sobre o ChatGPT dentro dessas restrições.

<br>

### Objetivo

O objetivo deste projeto é **analisar os sentimentos dos usuários no X (Twitter) em relação ao ChatGPT**. Utilizando dados já extraídos, o modelo irá identificar se a opinião predominante sobre o ChatGPT é positiva, negativa ou neutra. 

<br>

### Pergunta de Negócio Principal

> "**Qual é o sentimento predominante nas redes sociais, especificamente no X (Twitter), sobre o ChatGPT nesta última semana?**"

<br>

### Entregável

O entregável do projeto será uma **Análise utilizando a Biblioteca VADER**, que se baseia em uma abordagem baseada em regras para classificar os tweets como positivos, negativos ou neutros.

<br><br>

## Sobre o Conjunto de Dados

Os dados foram **extraídos em tempo real via Bardeen AI da plataforma X (anteriormente conhecida como Twitter)** no período de . As colunas principais de interesse incluem o conteúdo do tweet, o número de interações (retweets, likes, comentários, visualizações), e informações relacionadas ao perfil do usuário. O conteúdo textual (coluna "Post Body") será a principal variável utilizada para a análise de sentimentos.

<br>

<table> <tr> <th>Nome da Coluna</th> <th>Tipo de Dado</th> <th>Descrição</th> </tr> <tr> <td>Name</td> <td>object</td> <td>Nome do autor do tweet</td> </tr> <tr> <td>Handle</td> <td>object</td> <td>Nome de usuário (perfil) do autor do tweet</td> </tr> <tr> <td>Media URL</td> <td>object</td> <td>URL de qualquer mídia anexada ao tweet (imagem, vídeo)</td> </tr> <tr> <td>Retweets</td> <td>int64</td> <td>Número de retweets</td> </tr> <tr> <td>Likes</td> <td>object</td> <td>Número de likes no tweet</td> </tr> <tr> <td>Comments</td> <td>int64</td> <td>Número de comentários no tweet</td> </tr> <tr> <td>Views</td> <td>object</td> <td>Número de visualizações do tweet</td> </tr> <tr> <td>Tweet URL</td> <td>object</td> <td>Link direto para o tweet</td> </tr> <tr> <td>Profile Link</td> <td>object</td> <td>Link para o perfil do autor do tweet</td> </tr> <tr> <td>Post Body</td> <td>object</td> <td>Texto (conteúdo) do tweet</td> </tr> <tr> <td>Date</td> <td>object</td> <td>Data em que o tweet foi postado</td> </tr> <tr> <td>Timestamp</td> <td>object</td> <td>Registro de data e hora exata do tweet</td> </tr> </table>

<br><br><br><br><br>

# <span style="font-size: 40px; font-weight: bold;"><center>Instruções para executar o projeto</center></span>

### Criando os Containers Docker Para o Cluster Apache Kafka

1. Abrir o Docker Desktop.

2. Abra o prompt de comando ou terminal, navegue até a pasta onde estão os arquivos do projeto e execute o comando abaixo:

`docker-compose up -d`

Para desligar o cluster Kafka, execute:

`docker-compose down`

<br>

**Resumo**: Ao executar o comando `docker-compose up -d`, o Docker Compose irá iniciar e configurar os serviços definidos no arquivo `docker-compose.yml`. O comando inicia dois serviços, Zookeeper e Kafka, em containers Docker. Esses serviços são configurados para trabalhar juntos, com o Zookeeper gerenciando o Kafka, e o Kafka sendo capaz de criar e gerenciar tópicos de mensagens, como o `Analise_ChatGPT` que será criado na etapa seguinte. O `-d` faz com que os containers sejam executados em segundo plano (modo "detached"), permitindo que você continue usando o terminal para outros comandos. O nome que irá aparecer no Docker Desktop é o nome da rede padrão gerada pelo Docker Compose com base no nome do diretório (pasta) onde o arquivo `docker-compose.yml` está localizado.

<br><br><br><br>

# Importando Pacotes

<br>

In [2]:
# Manipulação de Dados
import numpy as np
import pandas as pd
import re

import re
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords') (Certifique-se de baixar os stopwords do NLTK se ainda não o fez)

# Tempo e Aleatoriedade
import time
import random

# Kafka e JSON

import kafka                       # Biblioteca para integração e manipulação com o Apache Kafka.
from json import dumps             # Função para converter objetos Python em strings JSON.
from kafka import KafkaProducer    # Classe para produzir mensagens para um tópico Kafka.

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Eduardo Brandão" --iversions

Author: Eduardo Brandão

numpy : 1.22.4
pandas: 1.3.5
kafka : 2.0.2
re    : 2.2.1
nltk  : 3.9.1



<br><br>

# 1. Configuração do Servidor Kafka e Tópico

In [4]:
# Endereço do servidor Kafka
SERVER = 'localhost:9092'

# Nome do tópico
TOPIC = "Analise_ChatGPT"

print('Etapa concluída.')

Etapa concluída.


**Explicação**: Define o endereço do servidor Kafka e o nome do tópico onde as mensagens serão enviadas.

<br><br>

# 2. Carregando e Preparando os Dados

In [5]:
# Carregamos o conjunto de dados de músicas
dados = pd.read_csv("dados/last_7_days_tweets_about_chatgpt.csv")

dados.head(2)

,Name,Handle,Media URL,Retweets,Likes,Comments,Views,Tweet URL,Profile Link,Post Body,Date,Timestamp
0,Jordan,@schtwiller,https://pbs.twimg.com/media/GXS9b-2bgAE5IZB?fo...,16,115,17,25 mil,https://x.com/schtwiller/status/18342998072148...,https://x.com/schtwiller,This is amazing. Used chatgpt o1-preview to cr...,12 de set,2024-09-12T18:35:28.000Z
1,Ozenua,@ozenua_,https://pbs.twimg.com/ext_tw_video_thumb/18362...,59,335,13,10 mil,https://x.com/ozenua_/status/1836329322044411926,https://x.com/ozenua_,Here's one way I use ChatGPT aside from creati...,11 h,2024-09-18T09:00:02.000Z


<br>

#### Criando Função para uma Análise Inicial

In [6]:
def funcao_analise_inicial(df):
    # Exibir o esquema do DataFrame
    print("\n\n INFO \n\n")
    print(df.info())

    # Número total de registros
    print(f"\nNúmero total de registros: {len(df)}")
    
    # Verificar valores ausentes
    print("\n\nVerificando valores ausentes:\n")
    df_ausentes = df.isnull().sum()
    print(df_ausentes[df_ausentes > 0])

    # Verificar valores duplicados
    num_linhas_duplicadas = df.duplicated().sum()
    porcentagem_linhas_duplicadas = (num_linhas_duplicadas / len(df)) * 100
    print(f"\n\nExistem valores duplicados: {num_linhas_duplicadas > 0}")
    if num_linhas_duplicadas > 0:
        print(f"Número de Linhas Duplicadas: {num_linhas_duplicadas}")
        print(f"Porcentagem de Linhas Duplicadas: {porcentagem_linhas_duplicadas:.2f}%")
    else:
        print("Nenhuma variável possui valores duplicados.")

    # Verificação de caracteres especiais
    caracteres_especiais = re.compile('[@_!#$%^&*<>()?/\\|}{~:]')
    colunas_com_caracteres_especiais = {}

    # Iterar sobre colunas do tipo string
    for coluna in df.select_dtypes(include=['object']).columns:
        contem_caracteres_especiais = df[coluna].apply(lambda x: bool(caracteres_especiais.search(str(x)))).any()
        if contem_caracteres_especiais:
            indices_com_caracteres_especiais = df[df[coluna].apply(lambda x: bool(caracteres_especiais.search(str(x))))][coluna].tolist()
            colunas_com_caracteres_especiais[coluna] = indices_com_caracteres_especiais

    # Exibe o resultado sobre caracteres especiais
    print("\n\nExistem caracteres especiais nas colunas:", bool(colunas_com_caracteres_especiais))
    if colunas_com_caracteres_especiais:
        print("\nColunas com caracteres especiais e exemplos de valores:")
        for coluna, indices in colunas_com_caracteres_especiais.items():
            print(f"\n Coluna [ {coluna} ]: Exemplo de valores com caracteres especiais: {indices[:5]}")
    else:
        print("\nNenhuma coluna possui caracteres especiais.")

print('A função foi criada com sucesso.\n\n')

A função foi criada com sucesso.




In [7]:
# Chama a Função Analise inicial
funcao_analise_inicial(dados)



 INFO 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          440 non-null    object
 1   Handle        440 non-null    object
 2   Media URL     256 non-null    object
 3   Retweets      406 non-null    object
 4   Likes         439 non-null    object
 5   Comments      419 non-null    object
 6   Views         440 non-null    object
 7   Tweet URL     440 non-null    object
 8   Profile Link  440 non-null    object
 9   Post Body     440 non-null    object
 10  Date          440 non-null    object
 11  Timestamp     440 non-null    object
dtypes: object(12)
memory usage: 41.4+ KB
None

Número total de registros: 440


Verificando valores ausentes:

Media URL    184
Retweets      34
Likes          1
Comments      21
dtype: int64


Existem valores duplicados: True
Número de Linhas Duplicadas: 10
Porcentagem de Linhas Duplicadas: 2.

<br>

#### Selecionando e Renomeando Colunas

In [8]:
# Selecionar as colunas 'Name', 'Likes', 'Views', 'Post Body' e 'Date'
dados_selecionados = dados[['Name', 'Likes', 'Views', 'Post Body', 'Timestamp']]

# Renomear a coluna 'Post Body' para 'Text'
dados_selecionados.rename(columns={'Post Body': 'Text'}, inplace=True)

# Excluir qualquer linha com valor NA
dados = dados_selecionados.dropna()

# Configura o Pandas para exibir todo o conteúdo da coluna sem truncar
pd.set_option('display.max_colwidth', None)

# Exibir todo o conteúdo da coluna 'Text'
display(dados.head(4))

,Name,Likes,Views,Text,Timestamp
0,Jordan,115,25 mil,This is amazing. Used chatgpt o1-preview to create an animation of a spinning donut in my terminal. I'm switching back to chatGPT from claude. This is the first try btw #ChatGPT #strawberry,2024-09-12T18:35:28.000Z
1,Ozenua,335,10 mil,Here's one way I use ChatGPT aside from creating landing page content...,2024-09-18T09:00:02.000Z
2,Avilasha Chalki,121,12 mil,#ChatGPT is down...how am I supposed to function,2024-09-18T06:59:53.000Z
3,Jafar Najafov,1 mil,221 mil,ChatGPT-4o is a GENIUS stock trader.\n\nBut most people don't know how to use it.\n\nHere are 10 prompts to use for stock trading and investing:,2024-09-14T05:10:02.000Z


<br>

#### Tratando Coluna `Text`

In [9]:
# Função para limpar o texto
def clean_text(text):
    """
    Limpa o texto removendo URLs, menções, emojis, pontuações,
    caracteres especiais (exceto hashtags) e converte para minúsculas.
    Substitui reticências por espaços.
    """
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove menções (@usuário)
    text = re.sub(r'\.{2,}', ' ', text)  # Substitui reticências por espaços
    text = re.sub(r'#', '', text)  # Remove apenas o símbolo '#' mantendo a palavra
    text = re.sub(r'[^\w\s\'#]', '', text)  # Remove emojis e pontuações, exceto apóstrofos e hashtags
    text = re.sub(r'\s+', ' ', text)  # Remove múltiplos espaços
    text = text.replace('_', '')  # Remove underlines manualmente
    text = text.lower().strip()  # Converte para minúsculas e remove espaços extras
    return text

# Função para remover stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    """
    Remove stopwords (palavras comuns) do texto para focar nas palavras mais relevantes.
    """
    words = text.split()
    filtered_text = ' '.join([word for word in words if word not in stop_words])
    return filtered_text

# Aplicar as funções de limpeza e remoção de stopwords no DataFrame do Pandas
dados['Text'] = dados['Text'].apply(clean_text)
dados['Text'] = dados['Text'].apply(remove_stopwords)

# Exibir as primeiras linhas após a aplicação das funções
display(dados.head(4))

,Name,Likes,Views,Text,Timestamp
0,Jordan,115,25 mil,amazing used chatgpt o1preview create animation spinning donut terminal i'm switching back chatgpt claude first try btw chatgpt strawberry,2024-09-12T18:35:28.000Z
1,Ozenua,335,10 mil,here's one way use chatgpt aside creating landing page content,2024-09-18T09:00:02.000Z
2,Avilasha Chalki,121,12 mil,chatgpt supposed function,2024-09-18T06:59:53.000Z
3,Jafar Najafov,1 mil,221 mil,chatgpt4o genius stock trader people know use 10 prompts use stock trading investing,2024-09-14T05:10:02.000Z


<br>

#### Tratando Colunas `Likes` e `Views`

In [10]:
# Função para converter valores com "mil", "milhão", ou valores numéricos com caracteres especiais
def converter_valores(text):
    # Substituir o espaço não quebrável por um espaço normal
    text = text.replace('\xa0', ' ').lower()
    
    # Verificar se o valor contém 'mil' ou 'milhão' e fazer as conversões adequadas
    if 'milhão' in text:
        return float(text.replace(' milhão', '').replace(',', '.')) * 1000000
    elif 'mil' in text:
        return float(text.replace(' mil', '').replace(',', '.')) * 1000
    else:
        # Tentativa de conversão direta para float, tratando casos de erro
        try:
            return float(text.replace(',', '.'))
        except ValueError:
            return pd.to_numeric(text, errors='coerce')

# Aplicar a função para tratar as colunas 'Likes' e 'Views'
dados['Likes'] = dados['Likes'].astype(str).apply(converter_valores)
dados['Views'] = dados['Views'].astype(str).apply(converter_valores)

print('Etapa concluída.')

Etapa concluída.


<br>

#### Removendo Linhas Duplicadas

In [11]:
# Verificar a quantidade de linhas duplicadas antes de remover
qtd_duplicadas_antes = dados.duplicated().sum()
print(f"Quantidade de linhas duplicadas antes de remover: {qtd_duplicadas_antes}")

# Remover as linhas duplicadas
dados = dados.drop_duplicates()

# Verificar a quantidade de linhas duplicadas novamente
qtd_duplicadas_depois = dados.duplicated().sum()
print(f"Quantidade de linhas duplicadas após remover: {qtd_duplicadas_depois}")

Quantidade de linhas duplicadas antes de remover: 13
Quantidade de linhas duplicadas após remover: 0


<br>

#### Criando Nova Coluna `Order_Id` e chama novamente `Função Análise Inicial`

In [12]:
# Cria uma coluna order_id com valores sequenciais
dados['Order_Id'] = np.arange(len(dados))

# Excluir linhas com valores ausentes
dados = dados.dropna()

# Reordenando as colunas
dados = dados[['Order_Id', 'Name', 'Likes', 'Views', 'Text', 'Timestamp']]

# Chama a Função Analise inicial
funcao_analise_inicial(dados)



 INFO 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 419 entries, 0 to 439
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Order_Id   419 non-null    int64  
 1   Name       419 non-null    object 
 2   Likes      419 non-null    float64
 3   Views      419 non-null    float64
 4   Text       419 non-null    object 
 5   Timestamp  419 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 22.9+ KB
None

Número total de registros: 419


Verificando valores ausentes:

Series([], dtype: int64)


Existem valores duplicados: False
Nenhuma variável possui valores duplicados.


Existem caracteres especiais nas colunas: True

Colunas com caracteres especiais e exemplos de valores:

 Coluna [ Name ]: Exemplo de valores com caracteres especiais: ['Skyler (KFR)', 'Łitecoin Bull | The News Before The News!', '@levelsio', 'Pangolin (V3 coming soon)', 'tut_ml']

 Coluna [ Timestamp ]: Exemplo de valores 


<br><br>

# 3. Conversão do DataFrame em Dicionário

In [13]:
# Convertemos o dataframe em um dicionário de músicas
dict_analises = dados.to_dict(orient = "records")
dict_analises[1:3]

[{'Order_Id': 1,
  'Name': 'Ozenua',
  'Likes': 335.0,
  'Views': 10000.0,
  'Text': "here's one way use chatgpt aside creating landing page content",
  'Timestamp': '2024-09-18T09:00:02.000Z'},
 {'Order_Id': 2,
  'Name': 'Avilasha Chalki',
  'Likes': 121.0,
  'Views': 12000.0,
  'Text': 'chatgpt supposed function',
  'Timestamp': '2024-09-18T06:59:53.000Z'}]

**Explicação**: Converte o DataFrame em uma lista de dicionários, onde cada dicionário representa uma música. Isso é necessário pois o Kafka armazena os dados no formato chave e valor.

<br><br>

# 4. Criação do Kafka Producer e Envio de Dados para o Kafka

In [14]:
# Kafka Producer
if __name__ == "__main__":

    # Cria o producer
    producer = KafkaProducer(bootstrap_servers = SERVER, 
                             value_serializer = lambda x: x.encode('utf-8'))
    
    # Variáveis de controle
    send = []    
    send = None

    # Loop pelo dicionário de músicas
    for tweet in dict_analises:
        
        # Cria a lista com dados que serão enviados para o Kafka
        sending = []
        
        # Append de cada coluna
        sending.append(tweet["Order_Id"])
        sending.append(tweet["Name"])
        sending.append(tweet["Likes"])
        sending.append(tweet["Views"])
        sending.append(tweet["Text"])
        sending.append(tweet["Timestamp"])
                
        # Junta tudo
        tweet = ','.join(str(v) for v in sending)

        # Envia os dados para o tópico
        print("Próximo Tweet:" )
        print(tweet)
        
        producer.send(TOPIC, tweet)
        time.sleep(8)                    # Remover o time.sleep(2) deixa o envio mais rápido

    print("Concluído")

Próximo Tweet:
0,Jordan,115.0,25000.0,amazing used chatgpt o1preview create animation spinning donut terminal i'm switching back chatgpt claude first try btw chatgpt strawberry,2024-09-12T18:35:28.000Z
Próximo Tweet:
1,Ozenua,335.0,10000.0,here's one way use chatgpt aside creating landing page content,2024-09-18T09:00:02.000Z
Próximo Tweet:
2,Avilasha Chalki,121.0,12000.0,chatgpt supposed function,2024-09-18T06:59:53.000Z
Próximo Tweet:
3,Jafar Najafov,1000.0,221000.0,chatgpt4o genius stock trader people know use 10 prompts use stock trading investing,2024-09-14T05:10:02.000Z
Próximo Tweet:
4,God of Prompt,286.0,45000.0,ive learned much using chatgpt4o didnt need pay courses 8 prompts use level fast,2024-09-18T04:40:02.000Z
Próximo Tweet:
5,SummerTime,49.0,1000.0,chatgpt magic internet money wzrd pretty well put,2024-09-12T01:21:25.000Z
Próximo Tweet:
7,Parul Gautam,91.0,25000.0,3 create review video ai access flikiai copy paste script written chatgpt video editing software review vide

Próximo Tweet:
51,kamran Hassan,472.0,142000.0,cancelled chatgpt subscription alternatives 10x better,2024-09-16T06:41:06.000Z
Próximo Tweet:
52,D-Coder,213.0,46000.0,chatgpt cheatsheet land dream job,2024-09-13T04:25:18.000Z
Próximo Tweet:
53,D-Coder,60.0,13000.0,never use chatgpt writing easily detectable try ai ensures text scream machine heres use,2024-09-18T05:37:09.000Z
Próximo Tweet:
54,I Meme Therefore I Am,50.0,502.0,stop using chatgpt pointless response start using common sense left,2024-09-17T13:53:07.000Z
Próximo Tweet:
55,Farzad,1000.0,63000.0,think grok outperform chatgpt end next year,2024-09-14T23:27:09.000Z
Próximo Tweet:
56,Linus ●ᴗ● Ekenstam,596.0,123000.0,chatgpt messaging people new upgrade rolling simulating level 2 intelligence full convo,2024-09-16T15:02:01.000Z
Próximo Tweet:
57,Geoff Lewis,49.0,3000.0,asked 7 randos shopping mall ai 57 answered chatgpt,2024-09-18T16:38:47.000Z
Próximo Tweet:
58,ChatGPT,5000.0,418000.0,welcome world openai o1,2024-09-12T17:26:3

Próximo Tweet:
102,Bojan Tunguz,5000.0,445000.0,phds thin chatgpt wrappers,2024-09-12T20:21:22.000Z
Próximo Tweet:
103,MOONDROP,125.0,5000.0,moonlight dances across night sky midautumn festival let magic moon drop gently heart celebrate joy cherish moments chatgpt happy midautumn festival moondrop,2024-09-17T02:00:29.000Z
Próximo Tweet:
104,Steven Heidel,84.0,5000.0,chatgpt soon single button press away,2024-09-18T18:26:11.000Z
Próximo Tweet:
105,Abhishek,1000.0,163000.0,chatgpt create mind maps wasting hundreds hours making visuals studying simplifying complex ideas heres free seconds,2024-09-14T13:44:42.000Z
Próximo Tweet:
106,Kanu Raj,67.0,13000.0,chatgpt old school try 8 alternatives blow mind bookmark later,2024-09-17T12:59:08.000Z
Próximo Tweet:
107,Nick Dobos,46.0,7000.0,5 minutes intrigued oddly works way chat part chatgpt works familiar interface,2024-09-17T04:34:09.000Z
Próximo Tweet:
108,Ihtesham Haider,477.0,114000.0,chatgpt4o moneymaking machine 5 chatgpt4o prompts make ri

Próximo Tweet:
152,VASU PRABHAS,46.0,574.0,chatgpt conversational ai model developed openai based gpt generative pretrained transformer architecture latest version interacting based gpt4 highly advanced model excels natural language understanding generation,2024-09-14T14:23:08.000Z
Próximo Tweet:
153,Tibor Blaho,58.0,4000.0,remember a8km123 tool codename tool used chatgpt o1preview shown thinking,2024-09-12T18:27:26.000Z
Próximo Tweet:
154,Ӿ ET Ӿ,57.0,2000.0,chatgpt's newest model o1 uses advanced reasoning used following prompt cryptocurrencies exist right one would make best global cash system take account user experience making choice response,2024-09-15T21:25:04.000Z
Próximo Tweet:
155,Andrew Bolis,2000.0,441000.0,i'm shocked people still can't make 297 every day using chatgpt like comment hustle ill dm detailed guide free follow get dm free next 24 hours,2024-09-15T08:15:02.000Z
Próximo Tweet:
156,Marko Terviö,325.0,55000.0,vow chatgpt o1preview aced intermediate microeconomics ex

Próximo Tweet:
195,Tulsi Soni,632.0,145000.0,chatgpt money making machine know use 10 ai sites guarantee pay rent every month,2024-09-17T04:49:47.000Z
Próximo Tweet:
196,Eyisha Zyer,74.0,14000.0,60 ai tools start profitable online business 2024 1 ideas claude chatgpt bing chat perplexity better research 2 website wegic ai unicorn 10web framer dora 3 writing rytr jasper longshot textblaze copymate 4 meeting tldv,2024-09-17T11:08:42.000Z
Próximo Tweet:
197,Adrian Dittmann,101.0,15000.0,openai's chatgpt o1mini fails strawberry xai's grok 2 beta gets right,2024-09-13T01:33:05.000Z
Próximo Tweet:
198,Oliver Jane,136.0,53000.0,chatgpt cheat sheet make money follow,2024-09-13T06:30:00.000Z
Próximo Tweet:
199,ilan,761.0,29000.0,onepiece1127 agenda piece mfs trying correct sandman chatgpt translations,2024-09-16T16:58:37.000Z
Próximo Tweet:
200,Naevis -77.82X-78.29,10000.0,471000.0,according chatgpt influential artists paved way kpop 1 seo taiji boys 1992 2 hot 1996 3 boa 2000s 4 tvxq 2003 5 bi

Próximo Tweet:
242,Lomez,1000.0,46000.0,watched first hour civil war last night prepared cringe hamfisted political conflict none prepared excruciatingly boring also cringe hamfisted drawn scenes journalists engaged chatgpt dialog,2024-09-17T17:11:14.000Z
Próximo Tweet:
243,tut_ml,510.0,41000.0,best ml courses machinelearning deeplearning bigdata datascience ml healthtech datavisualization artificialinteligence softwareengineering genai deeplearning chatgpt openai sql python ai ml gpt,2024-09-15T10:36:40.000Z
Próximo Tweet:
244,Apple Club,686.0,33000.0,ios 182 bring new features iphone december ios 182 expected december likely include features available current ios 18 betas apple intelligence including chatgpt integration localized additional englishspeaking countries image,2024-09-14T16:57:37.000Z
Próximo Tweet:
245,Tibor Blaho,97.0,13000.0,first two chatgpt system hints appear already rolled search find web picture use dalle,2024-09-12T06:59:08.000Z
Próximo Tweet:
246,Stocktwits,72.0

Próximo Tweet:
293,Youdontknowme{},85.0,7000.0,chatgpt response,2024-09-15T05:27:01.000Z
Próximo Tweet:
294,Brator𝕏,225.0,3000.0,videos people eating cats made chatgpt,2024-09-14T03:35:29.000Z
Próximo Tweet:
295,Prashant,102.0,13000.0,openai getting users switch back anthropic claude chatgpt,2024-09-12T17:31:56.000Z
Próximo Tweet:
296,Michael Di Stefano,148.0,9000.0,make sure chatgpt gets part,2024-09-13T11:02:12.000Z
Próximo Tweet:
297,65 Jellicles #SaveLowerDecks,58.0,830.0,explain words instead using chatgpt,2024-09-11T22:55:14.000Z
Próximo Tweet:
298,Chonkyy,61.0,1000.0,nice chatgpt got going,2024-09-14T21:12:12.000Z
Próximo Tweet:
299,Coral :3 (#1 CLAY BRICK FAN!!!),67.0,2000.0,thewboy done tons horrendous shit btw drawing peoples ocs extremely suggestively without permission using chatgpt write apology one drawing sexualized version beebo robot 64 mind minor,2024-09-14T11:12:30.000Z
Próximo Tweet:
300,A. M. Pines,165.0,5000.0,saying dont need chatgpt feel confident ability write 

Próximo Tweet:
346,Shruti Mishra,95.0,23000.0,literally chatgpt marketing professional marketer business used cost 5000month here's promote business spending 15 times less,2024-09-11T11:00:37.000Z
Próximo Tweet:
347,Łitecoin Bull | The News Before The News!,107.0,8000.0,chatgpt lol like going ebay buying fart trapped inside glass jar lol crapgpt figure haha,2024-09-12T22:47:36.000Z
Próximo Tweet:
348,William Fedus,1000.0,183000.0,happy release couple reasoning models today new models becoming larger contributor development future models many researchers engineers replaced large part chatgpt usage,2024-09-12T17:06:45.000Z
Próximo Tweet:
349,Subbarao Kambhampati (కంభంపాటి సుబ్బారావు),129.0,17000.0,unlike chatgpt make stupid errors confidence little computation o1 make stupid errors confidence significant thought charged account selfverification limitations maybe ever critical,2024-09-13T17:33:50.000Z
Próximo Tweet:
350,Michael Nielsen,128.0,11000.0,wrote 100 line emacspython script eddie

Próximo Tweet:
386,FELIX,289.0,53000.0,still using chatgpt 2024 that's outdated content usually easily recognizable avoid using chatgpt research could make research 10x seamless tool heres open,2024-09-13T04:11:57.000Z
Próximo Tweet:
387,Ayzore,68.0,5000.0,training llm takes ton energy actually using one sit chatgpt work day use less energy someone gaming 4090 day,2024-09-11T19:20:38.000Z
Próximo Tweet:
388,Clarissa Yorke,2000.0,258000.0,celebrate midautumn festival 3ex experience innovation chatgpt crypto exchanges free 3ex cryptogpt benefits await trading competition 3ex chatgpt giveaway cryptogpt btc ai trading,2024-09-13T04:05:25.000Z
Próximo Tweet:
389,TestingCatalog News,115.0,9000.0,chatgpt rolling system shortcuts support picture search options users type see popup selector available shortcut options selections trigger relevant chat experience based selection,2024-09-12T11:20:59.000Z
Próximo Tweet:
390,Geogristle,65.0,4000.0,chatgpt 4o seasoning,2024-09-13T02:41:51.000Z
Próximo

<br>

# Fim!